Project 1

The following two blocks of code were what I used to collect data and create a usable dataframe around it.

For my "documents," I went to 20 Wikipedia articles and parsed the first paragraph of each page using BeautifulSoup. I was having a difficult time getting scrapy to work on my computer. So, instead of crawling between webpages, I chose to enter the url for each manually. 

In [ ]:
###Used this code to generate my data sets###
from bs4 import BeautifulSoup
import requests
import pandas as pd

import subprocess as sp
sp.call('clear', shell = True)

response = requests.get("https://en.wikipedia.org/wiki/Cyber_Branch_(United_States_Army)")

if response is not None:
    html = BeautifulSoup(response.text, 'html.parser')
    paragraphs = html.select("p")
#    for para in paragraphs:
#        print (para.text)

    # just grab the text up to contents as stated in question
    intro = '\n'.join([ para.text for para in paragraphs[0:2]])
    print (intro)


values_for_csv = []
values_for_csv.append(intro)
values_for_csv = pd.DataFrame(values_for_csv)
values_for_csv.to_csv('raw_data.csv', mode = 'a', header = False)    

Below, I took the raw data and created a dataframe that placed each paragraph in the 'Title' column, followed by columns for each of the keywords I'd be searching for.

In [ ]:
import subprocess as sp
import pandas as pd
import numpy as np
sp.call('clear', shell = True)

#Import data#
raw_data = pd.read_csv('raw_data.csv', header = None)
del raw_data[0]
title = raw_data.values

#Create array with the keywords we'll be searching for.#
columns = ['Title','tech','computer','science','program','software',
           'internet','algorithm','data','computation','artificial','processing',
           'cyber','machine','digital','OUTCOME']
#Merge the titles with the empty columns into one data frame#
input_variables = np.zeros(shape = (len(raw_data.index),len(columns) - 1))
input_variables = np.concatenate([title,input_variables],axis = 1)
input_variables = pd.DataFrame(input_variables)
input_variables.columns = columns

input_variables.to_csv('training_data.csv')

At this point, all 20 entries are in the same dataframe. The block of code below takes the existing dataframe, prepares the data for Case 1 of the assignment, and splits the data into two new dataframes, of size 5 and 15 respectively. I included the Pandas' sampling function so I could quickly re-sample my test data. Since it was a small data set, I manually entered the values in the 'OUTCOME' column. I puposefully chose some topics that blurred the lines between mathematics and computer science. In those cases, I always stated the outcome as 0 'not CS.' so that I could infer how well the algorithm responded to mixed messages in the data.

In [1]:
import subprocess as sp
import pandas as pd
import numpy as np

sp.call('clear', shell = True)

data_train = pd.read_csv('training_data.csv')
del data_train['Unnamed: 0']
keywords = ['tech','computer','science','program','software',
           'internet','algorithm','data','computation','artificial','processing',
           'cyber','machine','digital']

data_train = data_train.sample(frac = 1).reset_index(drop = True)    
data_train['Title'] = data_train.Title.astype(str)

###Case 1###
for keyword in keywords:
    for title in data_train.index:
        my_string = data_train.loc[title,'Title'].lower()
        if my_string.find(keyword) != -1:
            data_train.loc[title,keyword] += 1

data_test = data_train.loc[0:4,:]
data_train = data_train.loc[5:,:].reset_index(drop = True)

data_train.to_csv('new_training_data.csv')
data_test.to_csv('test_data.csv')

Below is the code for Case 1.

In [2]:
#Column and Row labels for the data frames that will hold the conditional #
#probabilities#
keywords = ['tech','computer','science','program','software',
           'internet','algorithm','data','computation','artificial','processing',
           'cyber','machine','digital']
index = ['X=1|Y=1','X=0|Y=1','X=1|Y=0','X=0|Y=0']

#Import training set and initialize its dataframe of conditional probabilities#
data_train = pd.read_csv('new_training_data.csv')
del data_train['Unnamed: 0']

probabilities = np.zeros(shape = (4,len(data_train.index)-1))
probabilities = pd.DataFrame(probabilities,columns = keywords,
                                   index = index)

#Calculate the frequencies of each OUTCOME#
y_yes = int(data_train['OUTCOME'].agg(['sum']))
y_no = len(data_train['OUTCOME']) - y_yes
prob_y_yes = (y_yes/len(data_train.index))
prob_y_no = (y_no/len(data_train.index))


#Calculate conditional probabilities given OUTCOME is 1#
for column in range(len(probabilities.columns)):
    x_yes = x_no = 0
    for row in data_train.index:
        if data_train.iloc[row,-1] == 1:
            if data_train.iloc[row,column+1] == 1:
                x_yes += 1
            else:
                x_no += 1
    probabilities.iloc[0,column] = (x_yes/y_yes)
    probabilities.iloc[1,column] = (x_no/y_yes)
    
#Calculate conditional probabilities given OUTCOME is 0#
for column in range(len(probabilities.columns)):
    x_yes = x_no = 0
    for row in data_train.index:
        if data_train.iloc[row,-1] == 0:
            if data_train.iloc[row,column+1] == 1:
                x_yes += 1
            else:
                x_no += 1
        probabilities.iloc[2,column] = x_yes/y_no
        probabilities.iloc[3,column] = x_no/y_no
del(column,row,x_yes,x_no,index)

#Import test set and initialize its data frame of conditional probabilities#
data_test = pd.read_csv('test_data.csv')
del data_test['Unnamed: 0']

keywords = ['Probability Yes','Probability No', 'Final Decision']

probabilities_test = np.zeros(shape = (len(data_test.index),3))
probabilities_test = pd.DataFrame(probabilities_test,columns = keywords)

#Calculate probabilities of OUTCOME given the x's and compare to the true#
#probabilities#
#Probability that OUTCOME is 1#
for row in probabilities_test.index:
    conditional_prob = 1
    for column in range(1,14):
        if data_test.iloc[row,column] == 1:
            conditional_prob *= probabilities.iloc[0,column]
        else:
            conditional_prob *= probabilities.iloc[1,column]
    probabilities_test.iloc[row,0] = conditional_prob * prob_y_yes

#Probability that OUTCOME is 0#
for row in probabilities_test.index:
    conditional_prob = 1
    for column in range(1,14):
        if data_test.iloc[row,column] == 1:
            conditional_prob *= probabilities.iloc[2,column]
        else:
            conditional_prob *= probabilities.iloc[3,column]
    probabilities_test.iloc[row,1] = conditional_prob * prob_y_no

#Decide which OUTCOME to choose and calculate the model's accuracy#
percent_accuracy = 0
for row in probabilities_test.index:
    if probabilities_test.iloc[row,0] >= probabilities_test.iloc[row,1]:
        probabilities_test.iloc[row,-1] = 1
    else:
        probabilities_test.iloc[row,-1] = 0
    
    if probabilities_test.iloc[row,-1] == data_test.iloc[row,-1]:
        percent_accuracy += (1/len(probabilities_test.index))

Below is the training data set, and the outcomes for the test data. To save on space, I only printed the first 20 characters of each string. I'm including the .csv file of the raw data if you would like to see the entire paragraph for each row.

In [6]:
for row in data_train.index:
    my_string = data_train.iloc[row,0]
    data_train.iloc[row,0] = my_string[0:20]
display(data_train)
#print(data_train.head())
print("\n\n")

predicted_outcomes = pd.Series(probabilities_test['Final Decision'])
data_test['Predicted Outcomes'] = predicted_outcomes
for row in data_test.index:
    my_string = data_test.iloc[row,0]
    data_test.iloc[row,0] = my_string[0:20]
display(data_test[['Title','OUTCOME','Predicted Outcomes']])
#print(data_test[['Title','OUTCOME','Predicted Outcomes']])
print("Percent Error: ", (1 - percent_accuracy))


,Title,tech,computer,science,program,software,internet,algorithm,data,computation,artificial,processing,cyber,machine,digital,OUTCOME
0,The Cyber Branch is,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,The history of Artif,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,Statistics is the di,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,In machine learning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,Alan Mathison Turing,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
5,Software is programm,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,In computability the,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7,Computational scienc,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
8,Mathematics (from Gr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Computer science (so,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


,Title,OUTCOME,Predicted Outcomes
0,The philosophy of co,1.0,1.0
1,The Internet (portma,1.0,1.0
2,Experimental mathema,0.0,1.0
3,Sir Arthur Seymour S,0.0,0.0
4,From its beginnings,1.0,1.0


Percent Error:  0.19999999999999996


Below is the code for preparing the data for Case 2. Switching from the .find() to the .count() function is the only significant difference between this and the code I used for Case 1.

In [18]:
from scipy.stats import poisson

sp.call('clear', shell = True)

data_train = pd.read_csv('training_data.csv')
del data_train['Unnamed: 0']
keywords = ['tech','computer','science','program','software',
           'internet','algorithm','data','computation','artificial','processing',
           'cyber','machine','digital']

data_train = data_train.sample(frac = 1).reset_index(drop = True)    
data_train['Title'] = data_train.Title.astype(str)

###Case 2###
for keyword in keywords:
    for title in data_train.index:
        my_string = data_train.loc[title,'Title'].lower()
        data_train.loc[title,keyword] = my_string.count(keyword)

data_test = data_train.loc[0:4,:]
data_train = data_train.loc[5:,:].reset_index(drop = True)

data_train.to_csv('new_training_data.csv')
data_test.to_csv('test_data.csv')

Below is the code for Case 2. As you can see, I chose to the Poisson Distribution to calculate the conditional probabilities. The way I did this was:
1. I filtered the rows with 'OUTCOME' = 1
2. I took the mean of each keyword column
3. I used that mean as the $MLE(\lambda)$ for that variable
4. I stored the $\lambda$'s in a numpy array

I repeated the process when 'OUTCOME' = 0 and used each array in calculating the conditional probabilities for my test cases.

In [19]:
#Column and Row labels for the data frames that will hold the conditional #
#probabilities#
keywords = ['tech','computer','science','program','software',
           'internet','algorithm','data','computation','artificial','processing',
           'cyber','machine','digital']
index = ['X=1|Y=1','X=0|Y=1','X=1|Y=0','X=0|Y=0']

#Import training set#
data_train = pd.read_csv('new_training_data.csv')
del data_train['Unnamed: 0']


#Calculate the frequencies of each OUTCOME#
y_yes = int(data_train['OUTCOME'].agg(['sum']))
y_no = len(data_train['OUTCOME']) - y_yes
prob_y_yes = (y_yes/len(data_train.index))
prob_y_no = (y_no/len(data_train.index))

#Calculate the conditional probabilities given OUTCOME is 1#
yes_df = data_train[data_train['OUTCOME'] == 1]  
lam_yes = list()

for column in range(1,15):
    lam_yes.append(yes_df.iloc[:,column].mean())
lam_yes = np.array(lam_yes)

#Calculate the conditional probabilities given OUTCOME is 0#
no_df = data_train[data_train['OUTCOME'] == 0]
lam_no = list()

for column in range(1,15):
    lam_no.append(no_df.iloc[:,column].mean())
lam_no = np.array(lam_no)
del(yes_df,no_df,y_yes,y_no)

#Import test set and initialize its data frame of conditional probabiities#
data_test = pd.read_csv('test_data.csv')
del data_test['Unnamed: 0']

keywords = ['Probability Yes','Probability No', 'Final Decision']

probabilities_test = np.zeros(shape = (len(data_test.index),3))
probabilities_test = pd.DataFrame(probabilities_test,columns = keywords)

#Calculate probabilities of OUTCOME given the x's and compare to the true#
#probabilities#
#Probability that OUTCOME is 1#
for row in probabilities_test.index:
    conditional_prob = 1
    for column in range(1,14):
        if data_test.iloc[row,column] > 0:
            conditional_prob *= poisson.pmf(data_test.iloc[row,column],
                                            lam_yes[column])
        else:
            conditional_prob *= poisson.pmf(data_test.iloc[row,column],
                                            lam_no[column])
    probabilities_test.iloc[row,0] = conditional_prob * prob_y_yes

#Probability that OUTCOME is 0#
for row in probabilities_test.index:
    conditional_prob = 1
    for column in range(1,14):
        if data_test.iloc[row,column] == 0:
            conditional_prob *= poisson.pmf(data_test.iloc[row,column],
                                            lam_no[column])
        else:
            conditional_prob *= poisson.pmf(data_test.iloc[row,column],
                                            lam_yes[column])
    probabilities_test.iloc[row,1] = conditional_prob * prob_y_no

#Decide which OUTCOME to choose and calculate the model's accuracy#
percent_accuracy = 0
for row in probabilities_test.index:
    if probabilities_test.iloc[row,0] >= probabilities_test.iloc[row,1]:
        probabilities_test.iloc[row,-1] = 1
    else:
        probabilities_test.iloc[row,-1] = 0
    if probabilities_test.iloc[row,-1] == data_test.iloc[row,-1]:
        percent_accuracy += (1/len(probabilities_test.index))

In [20]:
for row in data_train.index:
    my_string = data_train.iloc[row,0]
    data_train.iloc[row,0] = my_string[0:20]
display(data_train)
print("\n\n")

predicted_outcomes = pd.Series(probabilities_test['Final Decision'])
data_test['Predicted Outcomes'] = predicted_outcomes
for row in data_test.index:
    my_string = data_test.iloc[row,0]
    data_test.iloc[row,0] = my_string[0:20]
display(data_test[['Title','OUTCOME','Predicted Outcomes']])
print("Percent Error", (1 - percent_accuracy))


,Title,tech,computer,science,program,software,internet,algorithm,data,computation,artificial,processing,cyber,machine,digital,OUTCOME
0,Alan Mathison Turing,0.0,4.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
1,"Technology (""science",8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,The Internet (portma,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Applied mathematics,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Maya civilizatio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,The Cyber Branch is,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
6,In machine learning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,In computability the,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
8,Experimental mathema,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Software is programm,0.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


,Title,OUTCOME,Predicted Outcomes
0,In mathematics and c,1.0,1.0
1,Sir Arthur Seymour S,0.0,0.0
2,The history of Artif,1.0,1.0
3,Computational scienc,1.0,0.0
4,An epidemic of 2019,0.0,0.0


Percent Error 0.19999999999999996


The data set I used is small, I chose the articles specifically, and a few keywords only appear in one article. Those are all reasons for this to be biased. However, I'm happy with the way the classifier turned out. Upon multiple runs of this code, it was not uncommon to get 60% error. As I watched this, I noticed it was certain "documents" that were difficult for the algorithm to classify. If my resampling of the dataframes placed that document in the test set, my error was certain to go up. I think with more data points, this algorithm could identify these "documents" more accurately.